# ADMN 5006 - Analytics for Financial Markets
# Assignment # 1

<p>Your assignment is to design and develop machine learning models for corporate bankruptcy prediction 2 years into the future using the data provided. </p>
   - The column “BK” in the data provided denotes whether the company goes bankrupt (indicated by 1) 2 years in the future, or not (indicated by 0). 
   - Some of the input variables included are “Assets Growth”, “Sales Growth”, “Earnings-per-share (EPS)”, “Return-on-equity (ROE)”, etc. 
   - In addition, the paper entitled: “Machine learning models and bankruptcy prediction” by Barboza et. al. is posted to the portal and is an excellent reference for you to use.  
    
<p>Note however that the dataset you are given is not the same as the one used in the aforementioned paper.  Therefore, don’t be alarmed if your results don’t agree with the Barboza et. al. paper.  You will need to partition your data into testing and training sets and explain your experimental set-up.</p>

<p>The input and output variables are described below:</p>
<table>
    <tr>
        <td>EPS</td>
        <td>Earnings Per Share</td>
    </tr>
    <tr>
        <td> Liquidity </td>
        <td> Working Capital/Total Assets</td>
    </tr>
    <tr>
        <td>Profitability</td>
        <td>Retained Earnings/Total Assets</td>
    </tr>
    <tr>
        <td>Productivity</td>
        <td>EBIT/Total Assets</td>
    </tr>
    <tr>
        <td>Leverage Ratio </td>
        <td>(Total Long-term debt + Debt in Current liabilities)/Stockholders Equity</td>
    </tr>
    <tr>
        <td>Asset Turnover</td>
        <td>Sales/ Total Assets</td>
    </tr>
    <tr>
        <td>Operational Margin</td>
        <td> EBIT/Sales</td>
    </tr>
    <tr>
        <td>Market Book Ratio</td>
        <td>(Price Close Annual Fiscal * Common Shares Outstanding)/Book Value Per Share</td>
    </tr>
    <tr>
        <td>Asset Growth</td>
        <td>Change in assets from previous year</td>
    </tr>
    <tr>
        <td>Sales Growth</td>
        <td>Change in sales from previous year</td>
    </tr>
    <tr>
        <td>Employee Growth</td>
        <td>Change in employees from previous year</td>
    </tr>
    <tr>
        <td>Tobin’s Q</td>
        <td>(Total market value of company + liabilities)/ (Total asset or book value + liabilities)</td>
    </tr>
    <tr>
        <td>BK</td>
        <td>Company bankrupt or not</td>
    </tr>
</table>

## Initialization
- Authors: Luke, Jad, Rita, Sreehari, Arnab, Varun
- Data: Bankruptcy_data_Final.xlsx
- Computing resources: Machine Learning (Random Forest, SVC, Naive Bayes)
- Software: Jupyter Notebook

## Import Libraries Required
The libraries listed are going to help us solving the dataset

In [34]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy import stats
import seaborn as sns

#for pre-processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer #for missing data values
from imblearn import under_sampling, over_sampling 
from imblearn.over_sampling import SMOTE #to normally distribute the data
from sklearn.neighbors import LocalOutlierFactor #for outliers

#computing resources
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score

## Import Data Source
In this part we have to import the data source to the notebook and see the basic statistics of the report.

- The data will be loaded using the pandas library which will make the excel file into a dataframe in python

In [3]:
#Data source: 
#Source Query location: 
path =  'Bankruptcy_data_Final.xlsx'
# reads the data from the file - denotes as CSV, it has no header, sets column headers
#df =  pd.read_csv(path, sep=',')
df = pd.read_excel(path)

## Describe Data
Now we apply the basic statistics of the dataset we got.
- Column Names
- Number of Rows and Columns
- Data types
- Descriptive Statistics
- Information
- Sample Dataset

In [4]:
df.columns

Index(['EPS', 'Liquidity', 'Profitability', 'Productivity', 'Leverage Ratio',
       'Asset Turnover', 'Operational Margin', 'Return on Equity',
       'Market Book Ratio', 'Assets Growth', 'Sales Growth', 'Employee Growth',
       'BK'],
      dtype='object')

In [5]:
df.shape

(92872, 13)

In [6]:
df.dtypes

EPS                   float64
Liquidity             float64
Profitability         float64
Productivity          float64
Leverage Ratio        float64
Asset Turnover        float64
Operational Margin    float64
Return on Equity      float64
Market Book Ratio     float64
Assets Growth         float64
Sales Growth          float64
Employee Growth       float64
BK                      int64
dtype: object

In [7]:
df.describe()

,EPS,Liquidity,Profitability,Productivity,Leverage Ratio,Asset Turnover,Operational Margin,Return on Equity,Market Book Ratio,Assets Growth,Sales Growth,Employee Growth,BK
count,92867.000000,92625.000000,92625.000000,92625.000000,92846.000000,92625.000000,87315.000000,92864.000000,9.281500e+04,86171.000000,86171.000000,85862.000000,92872.000000
mean,-14.461355,-2.631237,-29.537274,-1.222870,1.345316,1.053613,-7.915485,-2.112382,3.585063e+02,1.294075,1.900108,0.343322,0.006008
std,2195.467288,121.610921,677.230667,35.885556,253.038093,2.115945,214.460079,352.596902,2.606364e+04,73.769522,177.632638,14.074156,0.077280
min,-384000.000000,-25968.520000,-79682.000000,-5093.000000,-7811.000000,-31.590000,-30175.700000,-88875.140000,-3.151500e+06,-1.000000,-27.431034,-1.000000,0.000000
25%,-0.140000,0.020000,-0.640000,-0.060000,0.000000,0.390000,-0.030000,-0.080000,1.120000e+01,-0.052596,-0.033522,-0.048107,0.000000
50%,0.330000,0.190000,0.070000,0.060000,0.280000,0.830000,0.060000,0.030000,5.828000e+01,0.051844,0.059733,0.017241,0.000000
75%,1.530000,0.400000,0.310000,0.110000,0.820000,1.390000,0.140000,0.070000,2.401400e+02,0.192062,0.204231,0.130944,0.000000
max,55339.000000,1.000000,140.580000,1102.000000,75970.380000,276.380000,394.470000,39500.000000,3.455419e+06,14231.000000,39850.000000,2699.000000,1.000000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92872 entries, 0 to 92871
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   EPS                 92867 non-null  float64
 1   Liquidity           92625 non-null  float64
 2   Profitability       92625 non-null  float64
 3   Productivity        92625 non-null  float64
 4   Leverage Ratio      92846 non-null  float64
 5   Asset Turnover      92625 non-null  float64
 6   Operational Margin  87315 non-null  float64
 7   Return on Equity    92864 non-null  float64
 8   Market Book Ratio   92815 non-null  float64
 9   Assets Growth       86171 non-null  float64
 10  Sales Growth        86171 non-null  float64
 11  Employee Growth     85862 non-null  float64
 12  BK                  92872 non-null  int64  
dtypes: float64(12), int64(1)
memory usage: 9.2 MB


In [9]:
df.head()

,EPS,Liquidity,Profitability,Productivity,Leverage Ratio,Asset Turnover,Operational Margin,Return on Equity,Market Book Ratio,Assets Growth,Sales Growth,Employee Growth,BK
0,1.58,0.36,0.18,0.13,1.33,1.77,0.07,0.15,2.22,NaN,NaN,NaN,0
1,1.41,0.36,0.19,0.12,1.31,1.59,0.07,0.13,2.41,0.126319,0.014278,0.040179,0
2,0.31,0.32,0.13,0.08,1.03,1.55,0.05,0.04,2.56,0.368077,0.327909,0.566524,0
3,0.71,0.28,0.14,0.08,0.80,1.39,0.06,0.05,5.28,-0.020809,-0.118904,-0.095890,0
4,0.75,0.41,0.13,0.08,0.20,1.30,0.06,0.04,8.68,0.233089,0.146807,0.053030,0


## Data summary
 - Columns: 
        'EPS', 'Liquidity', 'Profitability', 'Productivity', 'Leverage Ratio',
       'Asset Turnover', 'Operational Margin', 'Return on Equity',
       'Market Book Ratio', 'Assets Growth', 'Sales Growth', 'Employee Growth',
       'BK'
 - Shape:
     (92872, 13)
 
 <p> Looking at the data there are 92872 rows and 13 columns in total </p>
       

## Check Data Validity
<p> To check our data validity; we need to check if there are missing values in the table, check of uniqueness, check for normality, and also check for extreme outliers in the dataset that could distort the algorithm </p>

In [10]:
#Lets start with missing values
def missing_values_table(df):
    #Code Description: Attempt to show the missing values in the table
    #Input: Dataframe
    #Output: missing value table rendered columns
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [11]:
missing_values_table(df)

Your selected dataframe has 13 columns.
There are 12 columns that have missing values.


,Missing Values,% of Total Values
Employee Growth,7010,7.5
Assets Growth,6701,7.2
Sales Growth,6701,7.2
Operational Margin,5557,6.0
Liquidity,247,0.3
Profitability,247,0.3
Productivity,247,0.3
Asset Turnover,247,0.3
Market Book Ratio,57,0.1
Leverage Ratio,26,0.0


If the columns have more than 50% missing data then we will drop the column but looking at the percentages we don't need to drop any of the columns

In [14]:
#Let's check for unique values in the columns
df.nunique()

EPS                    6987
Liquidity              6108
Profitability          9363
Productivity           6075
Leverage Ratio         6936
Asset Turnover         5051
Operational Margin     7485
Return on Equity       5750
Market Book Ratio     50336
Assets Growth         85604
Sales Growth          80350
Employee Growth       49532
BK                        2
dtype: int64

Looking at the dataset all the columns have more than 2 distinction. No need to drop any columns here

In [19]:
#Now lets delete rows which has more than 60% missing data
limit_per = int(len(df.columns) * .6)
df = df.dropna(thresh = limit_per)

In [20]:
df.shape

(92808, 13)

From 92872 rows, we have deleted 64 rows that has more than 7 columns that has no value in it.

## Adding data to missing values
Let's now try adding data to the missing values using the KNNImputer with n_neighbors = 5 
so we can check how effective is our model

In [22]:
imputer = KNNImputer(n_neighbors=5)
df_no_na = pd.DataFrame(imputer.fit_transform(df),columns=df.columns)
print("Missing Data Added")

Missing Data Added


In [23]:
df_no_na.isnull().sum()

EPS                   0
Liquidity             0
Profitability         0
Productivity          0
Leverage Ratio        0
Asset Turnover        0
Operational Margin    0
Return on Equity      0
Market Book Ratio     0
Assets Growth         0
Sales Growth          0
Employee Growth       0
BK                    0
dtype: int64

No more missing Data in our dataset

## Let's try applying some classification to our data

In [25]:
#let's divide our X coefficients to our Y 
#split df into X and Y
y = df_no_na['BK']
X = df_no_na.drop('BK',axis =1)

In [26]:
y

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
92803    0.0
92804    0.0
92805    0.0
92806    0.0
92807    1.0
Name: BK, Length: 92808, dtype: float64

In [27]:
X

,EPS,Liquidity,Profitability,Productivity,Leverage Ratio,Asset Turnover,Operational Margin,Return on Equity,Market Book Ratio,Assets Growth,Sales Growth,Employee Growth
0,1.580000,0.360000,0.180000,0.130000,1.330000,1.770000,0.070000,0.150000,2.220000,0.111757,-0.002385,0.057881
1,1.410000,0.360000,0.190000,0.120000,1.310000,1.590000,0.070000,0.130000,2.410000,0.126319,0.014278,0.040179
2,0.310000,0.320000,0.130000,0.080000,1.030000,1.550000,0.050000,0.040000,2.560000,0.368077,0.327909,0.566524
3,0.710000,0.280000,0.140000,0.080000,0.800000,1.390000,0.060000,0.050000,5.280000,-0.020809,-0.118904,-0.095890
4,0.750000,0.410000,0.130000,0.080000,0.200000,1.300000,0.060000,0.040000,8.680000,0.233089,0.146807,0.053030
...,...,...,...,...,...,...,...,...,...,...,...,...
92803,-1.488171,-0.015494,-0.758859,-0.056905,-1042.104918,0.174115,-0.326823,-6.614095,-1.846719,-0.073287,-0.557483,-0.076923
92804,-1.807630,0.093838,-1.204734,-0.121122,-4.530214,0.215844,-0.561155,-4.519074,-2.474549,-0.201835,-0.010544,-0.208333
92805,-0.015660,0.039260,0.000000,-0.082429,0.744907,0.254186,-0.324286,-0.569448,3274.505633,-0.168001,-0.020206,-0.105263
92806,-0.133054,0.053678,-0.029471,0.000998,0.574570,0.195683,0.005101,-0.085842,36.474842,0.076936,-0.170928,-0.058824


In [28]:
def preprocess_inputs(X,y):
    #train test split
    X_train,X_test,y_train,y_test= train_test_split(X, y, train_size = 0.7, shuffle = True, random_state = 1)
    
    # Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train),index=X_train.index, columns = X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test),index=X_test.index, columns = X_test.columns)
    return X_train, X_test, y_train, y_test

In [29]:
X_train, X_test, y_train, y_test = preprocess_inputs(X,y)

### Let's check our training variables

In [30]:
X_train

,EPS,Liquidity,Profitability,Productivity,Leverage Ratio,Asset Turnover,Operational Margin,Return on Equity,Market Book Ratio,Assets Growth,Sales Growth,Employee Growth
16454,0.008371,0.023963,0.046412,0.045285,-0.004624,0.005909,0.054164,0.008562,0.037793,-0.016716,-0.011214,-0.012562
30857,0.007625,0.022407,0.046291,0.041404,-0.005660,-0.183749,0.053691,0.008382,0.090556,-0.019603,-0.011724,-0.020604
91293,0.007531,0.021482,0.045333,0.035735,-0.006159,-0.439374,0.050388,0.008179,-0.013477,-0.022920,-0.011595,-0.018228
45022,0.007883,0.024529,0.046102,0.044176,-0.004557,-0.047690,0.054022,0.008433,-0.011227,-0.008849,-0.010155,0.038668
9079,0.008109,0.021558,0.045874,0.041959,-0.002452,-0.336299,0.054448,0.008485,-0.011037,-0.017967,-0.011944,-0.022698
...,...,...,...,...,...,...,...,...,...,...,...,...
21440,0.007422,0.023185,0.046210,0.038909,-0.004824,-0.022952,0.053123,0.008279,-0.006253,-0.018355,-0.011415,-0.016970
73349,0.007519,-1.667715,-1.553100,-3.461882,-0.007130,-0.418759,-12.624762,0.006791,-0.020783,-0.021575,-0.011595,0.078293
50057,0.007543,0.026438,0.042012,0.038078,-0.006161,-0.084797,0.052934,0.008228,-0.013071,-0.019339,-0.011057,-0.028954
5192,0.005242,0.025165,0.045685,0.041959,0.600197,0.286272,0.053549,0.005251,-0.009936,-0.022074,-0.012350,-0.050725


In [31]:
y_train

16454    0.0
30857    0.0
91293    0.0
45022    0.0
9079     0.0
        ... 
21440    0.0
73349    0.0
50057    0.0
5192     0.0
77708    0.0
Name: BK, Length: 64965, dtype: float64

### Apply Modelling techniques

In [32]:
models = {
    "Random Forest": RandomForestClassifier(),
    "SVC (RBF Kernel)": SVC(),
    "Naive Bayes" : GaussianNB()
}

In [33]:
for name, model in models.items():
    model.fit(X_train, y_train)
    print(name+ " trained.")

Random Forest trained.
SVC (RBF Kernel) trained.
Naive Bayes trained.


### Classification Metrics
To summarize classification metrics:
- **Accuracy** is a good measure to start with, if all classes are balanced
- **Precision** and **recall** becomes more important when classes are imbalanced.
- If false positive predictions are worse than false negatives, aim for higher precision.
- If false negative predictions are worse than false positives, aim for higher recall.
- **F1-score** is a combination of precision and recall

In [38]:
results = []
for name, model in models.items():
    result = model.score(X_test,y_test)
    results.append(result)
    print(name+ ": {:.2f}%".format(result*100))

Random Forest: 99.45%
SVC (RBF Kernel): 99.44%
Naive Bayes: 6.24%


We have checked the R^2 values for the 3 models:

Note: R^2 compares your models prediction to the mean of the targets. Values can range from negative infinity (a very ppor model) to 1. For example, if all your model does is predict the mean of thee targets, it's R^2 value would be 0. And if your model perfectly predicts a range of umbers it's R^2 value would be 1

In [40]:
score = []
for name, model in models.items():
    scores = cross_val_score(model,X_train,y_train, scoring='f1_macro',cv=10,n_jobs = 1)
    score.append(scores)
    print(f"{name}: {np.mean(score)}")

Random Forest: 0.500829140622062
SVC (RBF Kernel): 0.4996386763519428
Naive Bayes: 0.35398065605672624


In [47]:
def metrics(y_test,y_predh):
    from sklearn.metrics import accuracy_score,cohen_kappa_score,f1_score,log_loss,roc_auc_score, precision_score, recall_score
    print("Precision = {:.2f}".format(precision_score(y_test, y_predh)))
    print("Recall = {:.2f}".format(recall_score(y_test, y_predh)))
    print("F1 Score Macro = {:.2f}".format(f1_score(y_test, y_predh, average = 'macro')))
    print("AUC = {:.2f}".format(roc_auc_score(y_test, y_predh)))
    print("-----------------------------------------------------------")

In [48]:
import warnings
warnings.filterwarnings("ignore")
for name, model in models.items():
    y_pred = model.predict(X_test)
    print(name)
    print(classification_report(y_test,y_pred))
    metrics(y_test,y_pred)

Random Forest
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00     27687
         1.0       1.00      0.02      0.04       156

    accuracy                           0.99     27843
   macro avg       1.00      0.51      0.52     27843
weighted avg       0.99      0.99      0.99     27843

Precision = 1.00
Recall = 0.02
F1 Score Macro = 0.52
AUC = 0.51
-----------------------------------------------------------
SVC (RBF Kernel)
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00     27687
         1.0       0.00      0.00      0.00       156

    accuracy                           0.99     27843
   macro avg       0.50      0.50      0.50     27843
weighted avg       0.99      0.99      0.99     27843

Precision = 0.00
Recall = 0.00
F1 Score Macro = 0.50
AUC = 0.50
-----------------------------------------------------------
Naive Bayes
              precision    recall  f1-score   support



### Looks like we need to improve our output :(
Because our data is highly imbalanced lets try to apply re-sampling

In [ ]:
#Author1:
#Description: remove outliers from the dataset to give better results
# you can use LocalOutlierFactor to make it easier for you.
# be careful not to delete all the data because all the columns has outliers
# Severity: High

In [ ]:
#Author2:
#Description: use SMOTE to the training set to make the data balanced
#Severity: Low

In [ ]:
#Author3:
# Description: re apply the modelling techniques 
# I've made the modelling on top you can just follow what I did
# Severity: Medium

In [ ]:
#Author 4:
#Description: apply the metrics
# I made some examples on top in applying the metrics
#Severity: Medium

In [ ]:
#Author 5:
# description: Apply visualization and explain?
# Severity: Medium